In [4]:
import pandas as pd
from transformers import PreTrainedTokenizerFast
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch
import random

In [5]:
random.seed(0)

In [2]:
train_df = pd.read_csv('./train.csv')

In [92]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2', eos_token='</s>')
class GPTDataset(Dataset):
    def __init__(self,csv=pd.DataFrame):
        cont_word = [' 또한, ', ' 그리고 ']
        categories = ['마감재','인테리어','시공','마감하자','건축구조','기타','타 마감하자']
        # 데이터 포맷팅 및 토크나이징
        formatted_data = []
        for idx,(_, row) in tqdm(enumerate(csv.iterrows())):
            for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
                # 질문과 답변 쌍을 </s> token으로 연결
                input_text = row['질문_2'] 
                # 0.4 확률로 질문 연결
                if random.randint(1,10)<5:
                    next_q = csv[csv['category']==row['category']].drop(csv.index[idx]).sample(1).iloc[0,:]
                    input_text += (random.choice(cont_word)+next_q['질문_2']+tokenizer.eos_token +row[a_col]+' '+next_q[a_col])
                else:
                    input_text+= (tokenizer.eos_token +row[a_col])
                input_ids = tokenizer.encode(input_text, return_tensors='pt')[0].to(torch.int64)
                formatted_data.append(input_ids)
        print('Done.')
        self.data = formatted_data
        tokenizer.save_pretrained("./hansoldeco-kogpt2")
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        return self.data[idx]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [93]:
dataset = GPTDataset(train_df)

644it [00:02, 243.80it/s]

Done.
